In [2]:
from pyperplan import _parse, _ground
import numpy as np

In [3]:
problem = _parse(domain_file='modded_transport_pddl_files/6/domain.pddl',problem_file='modded_transport_pddl_files/6/p01.pddl')
task = _ground(problem)

## Naive embedding

- Naive binary embedding for a state space with N fluents is a N-dimensional binary vector that has a 1 for each true fact
- Naive continuous embedding is just the binary embedding times an NxM trainable embedding matrix
- Naive because we aren't using any shared feature representations for similar predicates/facts

In [4]:
def create_naive_embedding_fn(facts):
  """NOTE: Probably not best implementation atm; should be faster to initialize np.zeros, then 
  get indices of the state with a hash/dict, and set those indices to 1"""
  facts = list(facts)
  def naive_embedding(state):
    return np.array([(fact in state) for fact in facts], dtype=np.float32)
  return naive_embedding

In [5]:
naive_emb = create_naive_embedding_fn(task.facts)

In [6]:
# So the binary embedding of the initial state is:
naive_emb(task.initial_state)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [7]:
# We can now get an N-dimensional continuous embedding using a trainable embedding matrix
# This is equivalent to having an N-dimensional embedding for each "fact/fluent", and then adding them together.
N = 16
W_emb = np.random.normal(0., 1e-2, (len(task.facts), N))

# So now the initial embedding of the initial state is:
np.expand_dims(naive_emb(task.initial_state), 0).dot(W_emb)

array([[ 0.02025304, -0.11778014, -0.02690223, -0.06394005, -0.12289868,
        -0.03494775, -0.02668132, -0.06623775,  0.00317874,  0.01569206,
         0.00070324,  0.06517005, -0.00646302, -0.00898632,  0.01581173,
        -0.01682794]])

## Smarter embedding

- this is non-trivial... we should discuss